In [478]:
## Import relavant librarys

# Data librarys
import numpy as np
import pandas as pd

# Ploting library
import matplotlib.pyplot as plt
%matplotlib inline

In [479]:
def checkNAN(pandasdf):
    return pd.isnull(pandasdf).any(1).nonzero()[0]

In [499]:
## Readin CSV data using pandas 
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

m_data = train.shape[0]
m_test = test.shape[0]
train_valid_ratio = 0.8
print('Shape of the training data = '+str(train.shape))
print('Shape of the test data = '+str(test.shape))

## Append data together
print('type of the train data - ' + str(type(train)))
full = train.append(test, ignore_index=True)
print('Shape of the full data = ' + str(full.shape))

train = full


Shape of the training data = (891, 12)
Shape of the test data = (418, 11)
type of the train data - <class 'pandas.core.frame.DataFrame'>
Shape of the full data = (1309, 12)


In [500]:
train[m_data:]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
891,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
892,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
893,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
894,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
895,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298
896,14.0,NaN,S,9.2250,"Svensson, Mr. Johan Cervin",0,897,3,male,0,NaN,7538
897,30.0,NaN,Q,7.6292,"Connolly, Miss. Kate",0,898,3,female,0,NaN,330972
898,26.0,NaN,S,29.0000,"Caldwell, Mr. Albert Francis",1,899,2,male,1,NaN,248738
899,18.0,NaN,C,7.2292,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",0,900,3,female,0,NaN,2657
900,21.0,NaN,S,24.1500,"Davies, Mr. John Samuel",0,901,3,male,2,NaN,A/4 48871


In [501]:
train.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [529]:
## Transform Pclass
pclass = pd.get_dummies(train.Pclass,prefix='pclass')

## Transform Sex into male:feamle = 1:0
sex = pd.Series(np.where(train.Sex=='male',0,1),name='sex')
#sex = pd.get_dummies(train.Sex, )

## Tranform embarked type CQS into indices
embarked = pd.get_dummies(train.Embarked,prefix='embarked')

## Tranform cabin type
def getCabin(cab):
    if pd.isnull(cab):
        return 'Unknown'
    return cab[0]

cabin = train.Cabin#.fillna('U')
cabin = cabin.map(getCabin)
cabin= pd.get_dummies(cabin, prefix='Cabin')

#print(cabin)
## Transform famly size
family = train.Parch+train.SibSp+1
family = pd.DataFrame(np.where(family>1,1,0),columns=['family'])


In [530]:
## Obtain of people
name = train.Name
# we extract the title from each name
titles = name.map(lambda name: name.split(',')[1].split('.')[0].strip())
titles = pd.get_dummies(titles, prefix = 'titles')

In [531]:
## Transform ticket 
# only use the number in the ticket..
#   separate ticket in two stations and obained order based on number of digits
tickets = train.Ticket
def getTicketStation(ticket):
    tmp = ticket.split(' ')
    if len(tmp)>1:
        return tmp[0][0]
    else:
        return 'U_station'
def getTicketNumber(ticket):
    tmp = ticket.split(' ')
    if len(tmp)>1:
        return len(tmp[1])
    else:
        return len(tmp[0])
    
stations = pd.get_dummies(tickets.map(getTicketStation),prefix='station')
#np.where(tickets=='mp',1,0)
ticketNumbers = tickets.map(getTicketNumber)
ticketNumbers = pd.get_dummies(ticketNumbers, prefix='ticket')
#ticketnumbers
#stations


In [532]:
## Selecting a model from SKlearn
##
##
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

In [533]:
## Check for NAN
print(checkNAN(pclass), sex.isnull().all(),checkNAN(embarked),checkNAN(cabin), checkNAN(family), checkNAN(titles), \
      checkNAN(stations), checkNAN(ticketNumbers))

(array([], dtype=int64), False, array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [534]:
full_x = pd.concat([pclass, sex, embarked, cabin, family, titles, stations, ticketNumbers], axis=1)
m_train = int(m_data*train_valid_ratio)

train_x , valid_x , train_y , valid_y = train_test_split( full_x[0:m_data] , train.Survived[0:m_data] , \
                                                            train_size = train_valid_ratio )
#train_x = full_x[0:m_train]
#train_y = train.Survived[0:m_train]

#valid_x = full_x[m_train:m_data]
#valid_y = train.Survived[m_train:m_data]

#test_x = pd.DataFrame(data = full_x[m_data:m_data+m_test],index=np.arange(m_test))

print(train_x.shape, valid_x.shape, test_x.shape, train.shape)

((712, 50), (179, 50), (418, 50), (1309, 12))


In [535]:
# Fit the training set
model.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [536]:
### Yeah! my first training!
print (model.score( train_x , train_y ) , model.score( valid_x , valid_y ))

(0.824438202247191, 0.76536312849162014)


In [549]:
print('CheckNAN = ', checkNAN(test_x))
test_x = full_x.iloc[m_data:]
#test_x.iloc[0]
#train_x.iloc[0]
test_y = model.predict(test_x)

('CheckNAN = ', array([], dtype=int64))


In [553]:
test_passengerid = full[m_data:].PassengerId	
testOutput = pd.DataFrame({'PassengerID': test_passengerid, 'Survived':test_y})
testOutput.shape
testOutput.head()
testOutput.to_csv(name='ftl_titanic_submission.csv',index=False)

TypeError: to_csv() got an unexpected keyword argument 'name'